In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Loading libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from statistics import mean
from torch.autograd import Variable
import time
from PIL import Image
import random
from skimage import io, transform
from collections import OrderedDict
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, utils
from torch import nn

from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
from matplotlib import pyplot as plt
from skimage import io, transform
import skimage
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')



In [3]:
class ImageWithMaskDataset(Dataset):
    
    def __init__(self, csv_filePath, image_dir, mask_dir, transform=None):
        train_df = pd.read_csv(csv_file_path)
            
#         masks = os.listdir(mask_dir)
#         masks_df = pd.Series(masks).to_frame()
#         masks_df.columns = ['mask_file_name']
#         masks_df['image_id'] = masks_df.mask_file_name.apply(lambda x: x.split('_')[0])
#         train_df = pd.merge(train_df, masks_df, on='image_id', how='outer')
#         del masks_df
#         print(f"There are {len(train_df[train_df.mask_file_name.isna()])} images without a mask.")

#         ## removing items where image mask is null
#         train_df = train_df[~train_df.mask_file_name.isna()]        
        
#         images = os.listdir(image_dir)
#         images_df = pd.Series(images).to_frame()
#         images_df.columns = ['image_file_name']
#         images_df['image_id'] = images_df.image_file_name.apply(lambda x: x.split('.')[0])
#         train_df = pd.merge(train_df, images_df, on='image_id', how='outer')
#         del images_df 
#         print(f"There are {len(train_df[train_df.image_file_name.isna()])} image_ids without a image.")
        
#         ## Remove image_id not present in image_dir
#         train_df = train_df[~train_df.image_file_name.isna()]                
        
        self.data_frame = train_df.copy()
        
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        print('Creating dataset with {} examples'.format(len(self.data_frame)))
        
    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        
        
        image_id = self.data_frame.iloc[idx]['image_id']
        image_file_path = os.path.join(self.image_dir, image_id + ".tiff")
        image = skimage.io.MultiImage(image_file_path)[-1]
        
#         image = skimage.transform.resize(image, (224, 224))                
#         image = np.array(image)
        
        image = transform.resize(image,(256,256))
        image = image / 255
        image = image.transpose((2, 0, 1))
        
        
        
        try:
            mask_file_path = os.path.join(self.mask_dir, image_id + "_mask.tiff")
            mask = skimage.io.MultiImage(mask_file_path)[-1]
            
            mask = transform.resize(mask,(256,256))
            mask = mask / 255
    #         print('mask shape', mask.shape)
            mask = mask.transpose((2, 0, 1))        
            mask = mask[0,:,:]
            
#         mask = np.expand_dims(mask,axis=-1).transpose((2, 0, 1))
        except:
            return None
                        
        isup_grade = self.data_frame.iloc[idx]['isup_grade']

        return (torch.from_numpy(image), torch.from_numpy(mask), isup_grade)

In [4]:
def collate_fn(batch):
    batch = list(filter(lambda x: x is not None, batch))
    return torch.utils.data.dataloader.default_collate(batch)

In [5]:
image_dir = '/project/data/train_images'
mask_dir = '/project/data/train_label_masks'
csv_file_path = '/home/abharani/data/train_filtered.csv'

# csv_file_path = '/kaggle/input/train-filtered/train_filtered.csv'
# # csv_file_path =  '/kaggle/input/prostate-cancer-grade-assessment/train.csv'
# image_dir = '/kaggle/input/prostate-cancer-grade-assessment/train_images'
# mask_dir = '/kaggle/input/prostate-cancer-grade-assessment/train_label_masks'


dataset_with_mask = ImageWithMaskDataset(csv_file_path, image_dir, mask_dir)

for i in range(len(dataset_with_mask)):
    sample = dataset_with_mask[i]
    print(i, sample[0].shape, sample[1].shape)
    
    if i == 2:
        plt.show()
        break

Creating dataset with 10616 examples
0 torch.Size([3, 256, 256]) torch.Size([256, 256])
1 torch.Size([3, 256, 256]) torch.Size([256, 256])
2 torch.Size([3, 256, 256]) torch.Size([256, 256])


In [6]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models
import pandas as pd
import os
import skimage 
from skimage import io 
import skimage.transform

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset, WeightedRandomSampler
import torch.nn.functional as F
from torchvision import models

def train_val_dataset(dataset, val_split=0.25, generate_small=False):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets_created = {}
    if generate_small:
        print("Generating Small Train, Test Dataset")
        datasets_created['train'] = Subset(dataset, train_idx[0:200])
        datasets_created['test'] = Subset(dataset, val_idx[0:50])
    else:
        datasets_created['train'] = Subset(dataset, train_idx)
        datasets_created['test'] = Subset(dataset, val_idx)        
    return datasets_created



dataset_final = {}
datasets_created_trial_1 = train_val_dataset(dataset_with_mask,val_split=0.10,generate_small=False)
datasets_created_trial_2 = train_val_dataset(datasets_created_trial_1['train'],val_split=0.25,generate_small=False)

dataset_final['test'] = datasets_created_trial_1['test']
dataset_final['train'] = datasets_created_trial_2['train']
dataset_final['val'] = datasets_created_trial_2['test']

print("Dataset set size {}".format(len(dataset_with_mask)))
print("Train set size {}".format(len(dataset_final['train'])))
print("Validation set size {}".format(len(dataset_final['val'])))
print("Test set size {}".format(len(dataset_final['test'])))

Dataset set size 10616
Train set size 7165
Validation set size 2389
Test set size 1062


In [7]:
train_loader = DataLoader(dataset_final['train'],batch_size=32, collate_fn=collate_fn, shuffle = True, num_workers=4, pin_memory=True)
val_loader = DataLoader(dataset_final['val'],batch_size=32, num_workers=4, collate_fn=collate_fn,pin_memory=True)

dataloaders  = {'train': train_loader, 'val': val_loader}

dataset_sizes = {x: len(dataset_final[x]) for x in dataset_final.keys()}

In [8]:
# converting tensor to image
def image_convert(image):
    image = image.clone().cpu().numpy()
    image = image.transpose((1,2,0))
    image = (image * 255)
    return image

def mask_convert(mask):
    mask = mask.clone().cpu().detach().numpy()
    return np.squeeze(mask)

def plot_img(no_):
    iter_ = iter(train_loader)
    images,masks, _ = next(iter_)
    images = images.to(device)
    masks = masks.to(device)
    plt.figure(figsize=(20,10))
    for idx in range(0,no_):
         image = image_convert(images[idx])
         plt.subplot(2,no_,idx+1)
         plt.imshow(image)
    for idx in range(0,no_):
         mask = mask_convert(masks[idx])
         plt.subplot(2,no_,idx+no_+1)
         plt.imshow(mask,cmap='gray')
    plt.show()

In [9]:
# plot_img(1)

In [10]:
class ConvBlock(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size=(3,3),padding=1):
        super(ConvBlock,self).__init__()
        self.conv = nn.Conv2d(in_channels,out_channels,kernel_size,padding=padding,bias=False)
        self.batchnorm = nn.BatchNorm2d(out_channels,eps=1e-4)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self,x):
        x = self.conv(x)
        x = self.batchnorm(x)
        x = self.relu(x)
        return x
        
        
class StackEncoder(nn.Module):
    def __init__(self,channel1,channel2,kernel_size=(3,3),padding=1):
        super(StackEncoder,self).__init__()
        self.maxpool = nn.MaxPool2d(kernel_size=2,stride=2)
        self.block = nn.Sequential(
            ConvBlock(channel1,channel2,kernel_size,padding),
            ConvBlock(channel2,channel2,kernel_size,padding),     
        )
        
    def forward(self,x):
        big_out = self.block(x)
        poolout = self.maxpool(big_out)
        return big_out,poolout
     
        
class StackDecoder(nn.Module):
    def __init__(self,big_channel,channel1,channel2,kernel_size=(3,3),padding=1):
        super(StackDecoder,self).__init__()
        self.block = nn.Sequential(
            ConvBlock(channel1+big_channel,channel2,kernel_size,padding),
            ConvBlock(channel2,channel2,kernel_size,padding),
            ConvBlock(channel2,channel2,kernel_size,padding),
        )
        
    def forward(self,x,down_tensor):
            _, channels, height, width = down_tensor.size()  
            x = F.upsample(x, size=(height, width), mode='bilinear',align_corners=True)
            x = torch.cat([x, down_tensor], 1)  #combining channels of  input from encoder and upsampling input
            x = self.block(x)
            return x
        
        
class Unet256(nn.Module):
    def __init__(self,input_shape):
        super(Unet256,self).__init__()
        
        channel,height,width = input_shape
        
        self.down1 = StackEncoder(channel,12,kernel_size=(3,3))  #256
        self.down2 = StackEncoder(12,24,kernel_size=(3,3))  # 128
        self.down3 = StackEncoder(24,46,kernel_size=(3,3))  # 64
        self.down4 = StackEncoder(46,64,kernel_size=(3,3))  # 32
        self.down5 = StackEncoder(64,128,kernel_size=(3,3))  #16
        
        self.center = ConvBlock(128,128,kernel_size=(3,3),padding=1) #16
        
        self.up5 = StackDecoder(128,128,64,kernel_size=(3,3))  #32
        self.up4 = StackDecoder(64,64,46,kernel_size=(3,3)) #64
        self.up3 = StackDecoder(46,46,24,kernel_size=(3,3))
        self.up2 = StackDecoder(24,24,12,kernel_size=(3,3))
        self.up1 = StackDecoder(12,12,12,kernel_size=(3,3))
        self.conv = Conv2d(12,1,kernel_size=(1,1),bias=True)
        
    def forward(self,x):
        down1,out = self.down1(x)  
        down2,out = self.down2(out)  
        down3,out = self.down3(out)
        down4,out = self.down4(out)
        down5,out = self.down5(out)
        
        
        out = self.center(out)
        
        up5 = self.up5(out,down5)
        up4 = self.up4(up5,down4)
        up3 = self.up3(up4,down3)
        up2 = self.up2(up3,down2)
        up1 = self.up1(up2,down1)
        
        out = self.conv(up1)


        return out
    

In [14]:
## Testing the model dry run

In [17]:
x = torch.zeros((5, 3, 256, 256), dtype=torch.float32)  # minibatch size 64, image size [3, 32, 32]
model = Unet256((3,256,256))
scores = model(x)
print(scores.size())  # you should see [5,1,256,256]


torch.Size([5, 1, 256, 256])


In [11]:
# from torchsummary import summary
model = Unet256((3,256,256)).to(device)
# model

RuntimeError: CUDA error: out of memory

In [ ]:
class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)       
        bce_weight = 0.5
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
#         print("Inside DiceBCELoss inputs : {} , targets: {}".format(inputs.shape, targets.shape))
        
        intersection = (inputs * targets).sum()                            
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        loss_final = BCE * bce_weight + dice_loss * (1 - bce_weight)
        return loss_final
    
    

class IoU(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(IoU, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        #intersection is equivalent to True Positive count
        #union is the mutually inclusive area of all labels & predictions 
        intersection = (inputs * targets).sum()
        total = (inputs + targets).sum()
        union = total - intersection 
        
        IoU = (intersection + smooth)/(union + smooth)
                
        return IoU * 100

    
    
class DiceScore(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceScore, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice_score = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth) 
        return dice_score

In [ ]:
criterion = DiceBCELoss()


learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
epochs = 25


train_loss = []
val_loss = []

for epoch in range(epochs):
    print('Epoch {}/{}'.format(epoch + 1, epochs))
    start_time = time.time()
     

    
    running_train_loss = []
    
    for batch in train_loader: 
            image,mask,_ = batch[0], batch[1], batch[2]
            image = image.to(device,dtype=torch.float)
            mask = mask.to(device,dtype=torch.float)
            
            pred_mask = model.forward(image) # forward propogation
            
#             print("pred_mask : {}, mask : {} ".format(pred_mask.shape, mask.shape))
            loss = criterion(pred_mask,mask)
            optimizer.zero_grad() # setting gradient to zero
            loss.backward()
            optimizer.step()
            running_train_loss.append(loss.item())
                              

    else:           
        running_val_loss = []
        
        with torch.no_grad():
            for image,mask in val_loader:
                    image = image.to(device,dtype=torch.float)
                    mask = mask.to(device,dtype=torch.float)                            
                    pred_mask = model.forward(image)
                    loss = criterion(pred_mask,mask)
                    running_val_loss.append(loss.item())
                    

                                    
    
    epoch_train_loss = np.mean(running_train_loss) 
    print('Train loss: {}'.format(epoch_train_loss))                       
    train_loss.append(epoch_train_loss)
    
    epoch_val_loss = np.mean(running_val_loss)
    print('Validation loss: {}'.format(epoch_val_loss))                                
    val_loss.append(epoch_val_loss)
                      
    time_elapsed = time.time() - start_time
    print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

In [ ]:
plt.plot(train_loss,label='train_loss')
plt.plot(val_loss,label='val_loss')
plt.legend()
plt.title('Loss Plot')
plt.show()